In [ ]:
from pyspark.sql import functions as fns
from pyspark import  SparkContext
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [ ]:
from read_zips_from_s3 import get_zip_data_from_s3

In [ ]:
raw_data = get_zip_data_from_s3(sc)

In [ ]:
split_col = fns.split(raw_data['line'], ',')
raw_data = raw_data.withColumn('first_col', split_col.getItem(0))

# Deal with possible errors - if first_col is not two characters, both digits, then remove 
conformant_data = raw_data.filter(raw_data["line"].rlike("^\d{2},"))

In [ ]:
# Count number of records of different types.  
# TODO:  I think the metadatatable contains expected recordcounts?
conformant_data.createOrReplaceTempView("df")

spark.sql("select first_col, count(*) from df group by first_col").show()

In [ ]:
conformant_data.write.partitionBy("first_col").mode('overwrite').text("s3a://alpha-everyone/deleteathenaout/outtemp/")